In [2]:
import websocket
import json
import pandas as pd
from binance.client import Client
client = Client()

In [3]:
dict_ = client.get_exchange_info()

In [4]:
dict_

{'timezone': 'UTC',
 'serverTime': 1715603043070,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 6000},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 100},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 200000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 61000}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'otoAllowed': False,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelRe

In [5]:
syms = [i['symbol'] for i in dict_['symbols'] if i['symbol'].endswith('USDT')]

In [6]:
len(syms)

501

In [8]:
assets = [coin.lower()  +  '@kline_1m' for coin in syms]
relevant_assets = '/'.join(assets)

In [10]:
def manipulate(data):
    value_data = data['data']['k']
    price, sym = value_data['c'], value_data['s']
    event_time = pd.to_datetime([data['data']['E']], unit='ms')
    df = pd.DataFrame([[price, sym]],index=event_time)
    return df

In [12]:
def on_message(ws, message):
    json_message = json.loads(message)
    df_ =  manipulate(json_message)
    df_.to_csv('CoinPrices.csv', mode='a', header=False)

socket = "wss://stream.binance.com:9443/stream?streams=" + relevant_assets

ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

True